# ANEEL Public API Data

## Importing libraries

In [1]:
import requests
import polars as pl
from datetime import datetime as time

## Defining a class
Just to deal with the data

In [2]:
class DataHandler:
    def __init__(self):
        self.base_url = "https://dadosabertos.aneel.gov.br"
        self.result_total = self.get_result_total()

    
    def get_result_total(self) -> str:
        url = self.get_initial_url(0)
        answer = requests.get(url).json()
        return answer.get("result").get("total")

    
    def get_initial_url(self, limit: int) -> str:
        return f"{self.base_url}/api/3/action/datastore_search" + \
            f"?resource_id=b1bd71e7-d0ad-4214-9053-cbd58e9564a7&limit={limit}"

    
    def load_data(self) -> pl.LazyFrame:        
        url = self.get_initial_url(self.result_total)
        
        answer = requests.get(url).json()
        
        records = answer.get("result").get("records")
        records = [f"{record}\n" for record in records]
        
        return pl.scan_ndjson(records)

    def get_df(self, lf: pl.LazyFrame):
        lf
    
    # Currently not being needed.
    # In case pagination become attractive just use it.
    def load_data_batch(self,
        batch_size: int = 100,
        url: str = "",
        request_offset: int = 0,
        lazyframes: [pl.LazyFrame] = []
    ) -> pl.LazyFrame:


        init_time = datetime.datetime.now()
        if not url:
            url = self.get_initial_url(self.request_limit)
        
        
        number_of_records = request_offset + batch_size
        print(f"Loaded: {number_of_records}/{answer.get("result").get("total")}")

        
        answer = requests.get(url).json()
        
        records = answer.get("result").get("records")
        records = [f"{record}\n" for record in records]
        lazyframes.append(
            pl.scan_ndjson(records)
        )


        if (
            number_of_records >=
            answer.get("result").get("total")
        ):
            print(f"Time: {datetime.datetime.now() - init_time}")

            return pl.concat(lazyframes)

        else:            
            url = self.base_url + answer.get("result").get("_links").get("next")
            request_offset += self.request_limit


            print(f"Time: {datetime.datetime.now() - init_time}")
            self.load_data(url, request_offset, lazyframes)

## Putting it into work

In [3]:
initial_time = time.now()

dd = DataHandler()
first_request_time = time.now()

lf = dd.load_data()
final_time = time.now()

print(f"First request: {first_request_time - initial_time};\n" +
        f"Second request: {final_time - first_request_time};\n" +
        f"Total: {final_time - initial_time}")

First request: 0:00:00.814021;
Second request: 0:00:12.919415;
Total: 0:00:13.733436


## Code for manualy testing the API

In [4]:
BASE_URL = "https://dadosabertos.aneel.gov.br/api/3/action/datastore_search"

params = {
    "resource_id": 'b1bd71e7-d0ad-4214-9053-cbd58e9564a7',
    "limit": 2,
    "offset": 0,
}
r = requests.get(BASE_URL, params=params).json()

In [5]:
r#.get("next")#.json()#.get("result").get("total")

{'help': 'https://dadosabertos.aneel.gov.br/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 2,
  'offset': 0,
  'records_format': 'objects',
  'resource_id': 'b1bd71e7-d0ad-4214-9053-cbd58e9564a7',
  'total_estimation_threshold': None,
  'records': [{'_id': 1,
    'DatGeracaoConjuntoDados': '2025-06-09',
    'AnmPeriodoReferencia': '06/2025',
    'NumCNPJDistribuidora': '08336783000190',
    'SigAgente': 'CELESC-DIS',
    'NomAgente': 'CELESC DISTRIBUICAO S.A',
    'CodClasseConsumo': '3',
    'DscClasseConsumo': 'Rural',
    'CodSubGrupoTarifario': '10',
    'DscSubGrupoTarifario': 'B2',
    'CodUFibge': '42',
    'SigUF': 'SC',
    'CodRegiao': '4202',
    'NomRegiao': 'Sul',
    'CodMunicipioIbge': '4204509',
    'NomMunicipio': 'Corupá',
    'CodCEP': '89278***',
    'SigTipoConsumidor': 'PF',
    'NumCPFCNPJ': '***.371.499-**',
    'NomTitularEmpreendimento': '***',
    'CodEmpreendimento': '',
    'DthAtualizaCadastra